In [1]:
import pickle
import re
import string
import emoji
import pandas as pd
import numpy as np

from textblob import TextBlob
from collections import Counter

from langdetect import detect

import spacy

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF, LatentDirichletAllocation

import nltk
from nltk.tokenize import TweetTokenizer, word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer, WordNetLemmatizer

import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


**Kelsey 1-1**

- cleaning
    - preprocessing until comfortable with words
   
- sentiment analysis on all tweets
    - don't need to do any splitting at this stage
    - TextBlob & VaderSentiment first, spacy if the results aren't as expected
    
- topic modeling
    - decide: use all tweets (all topics) at once
        - start here
        - then can use these as features in the dataFrame and do splitting here
    - or: split to trump/biden - then bot/not bot for each
    - point here is there are multiple ways to split it
        - no right answers

In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_colwidth', None)

In [3]:
# must uncomment & run the first time to DOWNLOAD NLTK data
# I used package identifier 'popular'
# nltk.download()

In [4]:
df = pd.read_pickle("pickle/balanced_nov2_tweets.pick")
df.shape

(144000, 9)

In [5]:
df.columns

Index(['id', 'date', 'time', 'user_id', 'username', 'tweet', 'hashtags',
       'trump', 'biden'],
      dtype='object')

In [6]:
len(df.username.unique())

102600

In [7]:
df.sample(3)

,id,date,time,user_id,username,tweet,hashtags,trump,biden
67365,1323394849429770247,2020-11-02,22:41:34,847891925394960384,boycevida,@SUBRATA30016572 @JoeBiden https://t.co/NxBdVNalsR,[],False,True
71795,1323393507692814336,2020-11-02,22:36:14,979513121541967873,lindakost53,"Why Is Australia Reporting This Biden Child Porn Story, But The Media In America Hasn’t Reported It❓❓❓ Joe &amp; Hunter Biden Need To Be Investigated For Child Porn &amp; Sexual Assault Of Minors‼️",[],False,True
103198,1323383725716111363,2020-11-02,21:57:22,1203560977746276352,changed100times,@JoeBiden @KamalaHarris Where's Hunter Biden? can anyone tell me pleaseeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,[],False,True


Now let's create a subset, containing the same amount of Trump tweets as Biden tweets. We will exclude tweets that mention both candidates.

In [8]:
# keep only necessary columns
data = df.loc[:,['tweet', 'trump', 'biden', 'hashtags', 'user_id']]
data['original'] = data.tweet
data.drop(columns='tweet', inplace=True)
data.head(3)

,trump,biden,hashtags,user_id,original
181142,False,True,[],2820503362,"All these articles showing that Biden is in the lead.... IGNORE THAT AND STILL GO VOTE. All of these maps showing information that may or may not be correct won’t matter on Election Day. Hillary was also in the lead last election, just do your part. ⁽ᶠᵘᶜᵏ ᵗʳᵘᵐᵖ⁾"
0,False,True,[],1312487180258820096,@FoxNews Lady Gaga’s a nobody. Can’t figure out her own life and can’t even see nobody can help Biden. He’s out of the game from the day he gets in the game.
4,False,True,[],2335763630,"@The_Grupp “It is purely a fortuity that this isn’t one of the great mass casualty events in American history,” Ron Klain, who was Biden’s chief of staff at the time, said of H1N1 in 2019.” https://t.co/Umi317supK"


## Pre-Processing Pipeline

Now it's time to tokenize our tweets. Here are our pre-processing steps:
* Remove URLs
* Remove Twitter handles
* Remove numbers
* Convert to lowercase
* Remove punctuation
* Remove repeated letters so spell check will work ('aaaaand' -> 'aand')
* Remove non-English words
* Remove stop words

Since we're working with so many different words, I've chosen to use **lemmatization** instead of stemming for two reasons:
1. Lemmatization accurately reduces words to true meaning
2. Inxreased word reduction (handles synonyms)

In [ ]:
# Testing these before putting them in func


tweet = 'this is ! SAMPLE text...    @joebiden @donaldtrump #2020electionusa #2020ELECTIONUSA'

# remove urls
tweet = re.sub(r"https?:\/\/\S+", "", tweet)
# remove twitter handles
tweet = re.sub(r"@[\d\w_]+", "", tweet)
# remove numbers
tweet = re.sub('\w*\d\w*', ' ', tweet)
# convert to lowercase
tweet = re.sub('[%s]'.format(re.escape(string.punctuation)), ' ', tweet.lower())
# remove repeated letters so spell check will work (ex: 'aaaand' --> 'aand')
tweet = re.sub(r"([a-z])\1{2,}", r'\1', tweet)
# replace consecutive spaces with one
tweet = ' '.join(tweet.split())

tweet


In [10]:
# from autocorrect import Speller # TOO SLOW...TRY PYSPELLCHECKER
def tweet_tokenize(tweet, more_stop=None, more_words=None):
    """Get all of the tokens in a set of tweets.
    
    Parameters:
        - tweets (Series, required)
        
        - more_stop (List, optional): additional stop words to exclude
        
        - more_words (List, optional): additional words to INCLUDE in dictionary
    
    """
    # pre-processing pipeline
    
    # remove urls
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    # remove numbers
    tweet = re.sub('\w*\d\w*', ' ', tweet)
    # remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    tweet = tweet.lower()

    # and other popular campaign phrases
    tweet = re.sub(r"make america great again", "maga", tweet)
    tweet = re.sub(r"makeamericagreatagain", "maga", tweet)
    tweet = re.sub(r"sleepyjoe", "sleepy joe", tweet)
    tweet = re.sub(r"sleepyjoebiden", "sleepy joe biden", tweet)
    
    # remove spaces in candidate names
    tweet = re.sub(r"joebiden", "joe_biden", tweet)
    tweet = re.sub(r"kamalaharris", "kamala_harris", tweet)
    tweet = re.sub(r"donaldtrump", "donald_trump", tweet)
    tweet = re.sub(r"mikepence", "mike_pence", tweet)
    tweet = re.sub(r"joe biden", "joe_biden", tweet)
    tweet = re.sub(r"kamala harris", "kamala_harris", tweet)
    tweet = re.sub(r"donald trump", "donald_trump", tweet)
    tweet = re.sub(r"mike pence", "mike_pence", tweet)
    # replace 'biden' with 'joebiden' (do for all candidates)
    tweet = re.sub(r"\bbiden\b", "joe_biden", tweet)
    tweet = re.sub(r"\bpence\b", "mike_pence", tweet)
    tweet = re.sub(r"\bharris\b", "kamala_harris", tweet)
    tweet = re.sub(r"\btrump\b", "donald_trump", tweet)
    
    # remove repeated letters so spell check will work (ex: 'aaaand' --> 'aand')
    tweet = re.sub(r"([a-z])\1{2,}", r'\1', tweet)
    # replace consecutive spaces with one
    tweet = ' '.join(tweet.split())
    
    more_words = ['trump', 'biden', 'maga', 'bidenharris', 
                  'kamala', 'pence', 'harris', 'mike',
                  'bidenharris2020', 'trumppence',
                  'trumppence2020', 'usa', 'election2020',
                  'ivoted', 'joe_biden', 'realdonaldtrump',
                  'donald_trump', 'sleepy_joe',
                  'mike_pence', 'kamala_harris']
    
    dictionary = list(words.words()) + more_words
    dictionary = set(dictionary)
    
    twt = TweetTokenizer()
    tokens = [token for token in twt.tokenize(tweet) if token in dictionary]
    
    # initiate stop word removal and lemmatization    
    more_stop = ['fxhedg','fyck','fy','fxxking','give','go',
                 'going','gonna','get','one','de','la','el','en','un','ha',
                 'would','dont','know','time','think','want','via','dont']
    
    stop_words = list(stopwords.words('english')) + more_stop
    stop = stop_words
    stop = set(stop)
    
    lemm = WordNetLemmatizer()
    
    # implement lemmatization and stop word removal
    tokens = [lemm.lemmatize(token) for token in tokens
              if token.lower() not in stop]
#     spell = Speller(lang='en')
#     tokens = [spell(t) for t in tokens]

    combined_tokens = ' '.join(tokens)

    return combined_tokens

In [ ]:
data['tweet'] = data['original'].map(tweet_tokenize)

In [ ]:
data.to_pickle("pickle/n2_tokenized")

In [ ]:
data.sample(10)

In [ ]:
# data.to_pickle("pickle/tweets_df_5000tw.pick")

## Count Vectorizer

In [ ]:
cv = CountVectorizer(min_df=0.02, max_df=0.10)
doc_words = cv.fit_transform(data.tweet)

In [ ]:
data.iloc[5]

## NMF

In [ ]:
nmf_model = NMF(n_components=5)
doc_topic = nmf_model.fit_transform(doc_words)
print(f"Shape: {doc_topic.shape}")
print(f"Number of iterations used: {nmf_model.n_iter_}")

From lecture: The **doc_topic** matrix shows us the documents we started with, and how each document is made up of the 2 resulting topics. We don't know yet what the topics are.

In [ ]:
topic_word = nmf_model.components_
topic_word

In [ ]:
topic_word.argsort()

In [ ]:
topic_word.argsort()[:,-1:-7:-1]

From lecture: The **topic_word** matrix shows us the 2 resulting topics, and the terms that are associated with each topic. By looking at the words below, we an figure out what the topics are.


In [ ]:
words = cv.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

In [ ]:
doc_topic

## LDA

In [ ]:
lda_model = LatentDirichletAllocation(n_components=2)
doc_topic = lda_model.fit_transform(doc_words)
doc_topic.shape

In [ ]:
words = cv.get_feature_names()
t = lda_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

## Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [ ]:
sid_obj = SentimentIntensityAnalyzer()
sentiment = []
for text in data.tweet:
    sentiment.append(sid_obj.polarity_scores(text))
    
pd.concat([data,pd.DataFrame(sentiment)], axis=1)